In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *

In [0]:
spotify_table_0 = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferschema", "true")\
    .load("/FileStore/shared_uploads/arraz7191@gmail.com/spotify_data.csv")

In [0]:
spotify_table = spotify_table_0.dropna()

In [0]:
spotify_table.show()

+--------------------+--------------------+--------------------+-----------+------------+----+------------+------------+-------+----------------+--------+--------+-----------+-------+-------+----+---+----------+--------+
|                  id|                name|             artists|duration_ms|release_date|year|acousticness|danceability| energy|instrumentalness|liveness|loudness|speechiness|  tempo|valence|mode|key|popularity|explicit|
+--------------------+--------------------+--------------------+-----------+------------+----+------------+------------+-------+----------------+--------+--------+-----------+-------+-------+----+---+----------+--------+
|6KbQ3uYMLKb5jDxLF...|Singende Bataillo...| ['Carl Woitschach']|     158648|        1928|1928|       0.995|       0.708|  0.195|           0.563|   0.151| -12.428|     0.0506|118.469|  0.779|   1| 10|         0|       0|
|6KuQTIu1KoTTkLXKr...|Fantasiestücke, O...|['Robert Schumann...|     282133|        1928|1928|       0.994|       0.

##Filtrar canciones por año

In [0]:
def año_cancion(año):
    cancion_año = spotify_table.select("name", "year")
    cancion_año.filter(col("year") == año).show(truncate = False)

In [0]:
año_cancion(1980)

+-------------------------------------------+----+
|name                                       |year|
+-------------------------------------------+----+
|Meet Me In The City - Studio Outtake - 1979|1980|
|By Your Side                               |1980|
|Puerto de Illusion                         |1980|
|Devil in My Car                            |1980|
|Right Start - Unfinished Outtake           |1980|
|Where Were You                             |1980|
|My Only Love                               |1980|
|The World's a Mess, It's in My Kiss        |1980|
|Indian Girl - Remastered                   |1980|
|Shining                                    |1980|
|The Plastic Age                            |1980|
|Falling In Love                            |1980|
|Something About England - Remastered       |1980|
|I'm Losing You                             |1980|
|The Trumpet of Jesus                       |1980|
|Oh Well (Pt. 1) - Live 1980, St. Louis, MO |1980|
|Biscuits in the Oven          

##Canción más larga

In [0]:
spotify_table = spotify_table.withColumn("minutes", expr("duration_ms")/60000)
tiempo_cancion = spotify_table.select("name", "minutes")
tiempo_cancion = spotify_table.groupBy("name").agg(
    max(col("minutes")).alias("longer_song")
).orderBy(col("longer_song").desc()).show(truncate= False)

+--------------------------------------------------------------------------------------------------------------+------------------+
|name                                                                                                          |longer_song       |
+--------------------------------------------------------------------------------------------------------------+------------------+
|Brown Noise - 90 Minutes                                                                                      |90.05833333333334 |
|Brown Noise for Sleep                                                                                         |71.16723333333333 |
|Midnight Thunderstorm Part 2                                                                                  |71.15678333333334 |
|Ocean Waves Sounds                                                                                            |68.67096666666667 |
|Ocean Waves for Sleep                                                      

##Canciones más populares

In [0]:
popularidad = spotify_table.select("name", "popularity")
popularidad = spotify_table.groupBy("name").agg(
    max(col("popularity")).alias("Note")
).orderBy(col("Note").desc()).show(truncate = False)

+-------------------------------------------------------------------------------------------------+------+
|name                                                                                             |Note  |
+-------------------------------------------------------------------------------------------------+------+
|"Not Tonight (feat. Da Brat, Left Eye, Missy ""Misdemeanor"" Elliott and Angie Martinez) - Remix"|99.429|
|ROCKSTAR (feat. Roddy Ricch)                                                                     |99    |
|South Pacific (1949): A Wonderful Guy - Voice                                                    |97.724|
|death bed (coffee for your head) (feat. beabadoobee)                                             |97    |
|I Ain't Superstitious                                                                            |96.361|
|THE SCOTTS                                                                                       |96    |
|ily (i love you baby) (feat. Emilee)

##Promedio de acústica por año

In [0]:
acustica = spotify_table.select("year", "acousticness")
acustica = spotify_table.groupBy("year").agg(
    avg(col("acousticness")).alias("acustica")
).orderBy(col("acustica").desc())

acustica = acustica.withColumnRenamed("year", "canciones")

acustica.show(truncate = False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|canciones                                                                                                                                                                                           |acustica|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|['Johann Sebastian Bach', 'Otto Klemperer', 'Philharmonia Orchestra']                                                                                                                               |706400.0|
|['Ludwig van Beethoven', 'Jenő Jandó']                                                                                                                                 

##Números de canciones por artista

In [0]:
artista = spotify_table.groupBy("artists").count().orderBy(col("count").desc())
artista = artista.withColumnRenamed("count", "Num_canciones").show(truncate = False)

+------------------------------+-------------+
|artists                       |Num_canciones|
+------------------------------+-------------+
|['Эрнест Хемингуэй']          |1215         |
|['Francisco Canaro']          |938          |
|['Эрих Мария Ремарк']         |781          |
|['Ignacio Corsini']           |620          |
|['Frank Sinatra']             |592          |
|['Bob Dylan']                 |539          |
|['The Rolling Stones']        |512          |
|['Johnny Cash']               |502          |
|['The Beach Boys']            |491          |
|['Elvis Presley']             |488          |
|['Francisco Canaro', 'Charlo']|459          |
|['Queen']                     |426          |
|['Dean Martin']               |411          |
|['The Beatles']               |410          |
|['Miles Davis']               |408          |
|['Fleetwood Mac']             |398          |
|['Billie Holiday']            |395          |
|['Ella Fitzgerald']           |354          |
|['Lead Belly